In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# def check_same_quartier(df1, df2, col_nbquartier_df1, col_nbquartier_df2, col_namequartier_df1, col_namequartier_df2):
def check_same_quartier(df1, df2):
    for row in df1.itertuples():
        curr_no_qr = row.no_qr
        curr_nom_qr = row.nom_qr
        is_same = curr_nom_qr == df2[df2['no_qr'] == curr_no_qr]['nom_qr']
        if is_same.bool()==False:
            print(curr_no_qr, curr_nom_qr, df2[df2['no_qr'] == curr_no_qr]['nom_qr'])

# check_same_quartier(df_nb_logements, df_quartier)

In [ ]:
# Importation des quartiers
df_quartier = pd.read_csv('Data/quartierreferencehabitation.csv')
df_quartier.sort_values(by=['no_qr'], inplace=True)

# Importations du nombre de logements par quartier
df_nb_logements = pd.read_csv('Data/nombrelogementsquartiersreference.csv')
df_nb_logements.rename(columns={'No_QR': 'no_qr', 'Nom_QR': 'nom_qr'}, inplace=True)
df_nb_logements.sort_values(by=['no_qr'], inplace=True)

# Importation du nombre de logements sociaux par quartier
df_nb_log_sociaux = pd.read_csv('Data/logements-sociaux.csv', sep=';', encoding='latin-1')
df_nb_log_sociaux.rename(columns={'qr': 'no_qr'}, inplace=True)

final_df = pd.DataFrame()

df_quartier[df_quartier['nom_mun'] == 'Montréal']
final_df['no_qr'] = df_quartier[df_quartier['nom_mun'] == 'Montréal']['no_qr']
final_df['nom_qr'] = df_quartier[df_quartier['nom_mun'] == 'Montréal']['nom_qr']

In [ ]:
# Ajout du nombre d'habitants
nb_logements = []
for curr_no_qr in final_df['no_qr']:
    a = df_nb_logements[df_nb_logements['no_qr'] == curr_no_qr]['Nb_log'].tolist()
    nb_logements.append(a[0])
final_df['nb_logements'] = nb_logements

# Ajout du nombre de logements sociaux
nb_log_sociaux = []
for curr_no_qr in final_df['no_qr']:
    a = df_nb_log_sociaux[df_nb_log_sociaux['no_qr'] == curr_no_qr]['nb_log'].tolist()
    nb_log_sociaux.append(sum(a))
final_df['nb_log_sociaux'] = nb_log_sociaux

# Ajout proportion logements sociaux
a = final_df['nb_logements'].tolist()
b = final_df['nb_log_sociaux'].tolist()
prop_log_sociaux = np.divide(np.array(b), np.array(a)) * 100
final_df['prop_log_sociaux'] = prop_log_sociaux

In [ ]:
df_final = pd.read_csv('Data/bbugs_final.csv')
df_clean = pd.read_csv('Data/bbugs_clean.csv')
df_climate = pd.read_csv('Data/climate_per_week_final.csv')
df_climate = df_climate[13:]
cpt = 0

for id_qr in df_final.NO_QR.unique():
    
    # rows for the qr in df final
    df_final_qr = df_final.loc[df_final.NO_QR==id_qr]
    n_row, _ = df_final_qr.shape
    
    # nb logements and prop log sociaux for the qr
    final_df_qr = final_df.loc[final_df.no_qr==id_qr][['nb_logements','prop_log_sociaux']]
    final_df_qr_repeat = pd.concat([final_df_qr]*n_row, ignore_index=True).values
    
    # insert the 2 features to the data
    df_final_qr.insert(4,'prop_log_sociaux', final_df_qr_repeat[:,1], allow_duplicates = False)
    df_final_qr.insert(4,'nb_logements', final_df_qr_repeat[:,0], allow_duplicates = False)
    
    # centroid coordinates for the qr
    df_clean_qr = np.array(list([df_clean.loc[df_clean.no_qr==id_qr][['CENTROID_X', 'CENTROID_Y']].values[1]])*n_row)
    # insert centroids
    df_final_qr.insert(4,'CENTROID X', df_clean_qr[:,0], allow_duplicates = False)
    df_final_qr.insert(4,'CENTROID Y', df_clean_qr[:,1], allow_duplicates = False)
    
    if cpt == 0:
        
        DF_final = df_final_qr
    
    else:
        
        DF_final = pd.concat([DF_final, df_final_qr], ignore_index=True)
    
    cpt += 1

DF_final['NB_EXTERMINATIONS'] = DF_final['NB_EXTERMINATIONS']/DF_final['nb_logements']
DF_final['NB_EXTERMINATIONS_2'] = DF_final['NB_EXTERMINATIONS_2']/DF_final['nb_logements']
DF_final['NB_EXTERMINATIONS_3'] = DF_final['NB_EXTERMINATIONS_3']/DF_final['nb_logements']
DF_final['NB_EXTERMINATIONS_4'] = DF_final['NB_EXTERMINATIONS_4']/DF_final['nb_logements']
DF_final['LABEL_1mois'] = DF_final['LABEL_1mois']
DF_final['LABEL_3mois'] = DF_final['LABEL_3mois']
DF_final

In [ ]:
def trsnform_week_id(weid):
    b = weid.split('-')
    return float(int(b[0])*100 + int(b[1]))
NEW_WEEK_ID = []
for weid in DF_final['WEEK_ID']:
    NEW_WEEK_ID.append(trsnform_week_id(weid))
DF_final['WEEK_ID'] = NEW_WEEK_ID

In [ ]:
DF_FINAL = DF_final.merge(df_climate, left_on = 'WEEK_ID', right_on = 'id_week')

Y_FINAL = DF_FINAL[['LABEL_1mois', 'LABEL_3mois']]
cols_X = ['NO_QR', 'CENTROID Y', 'CENTROID X', 'nb_logements', 'prop_log_sociaux',
       'NB_EXTERMINATIONS', 'NB_EXTERMINATIONS_2', 'NB_EXTERMINATIONS_3',
       'NB_EXTERMINATIONS_4',
       'id_week', 'MEAN_TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE',
       'TOTAL_PRECIPITATION', 'TOTAL_RAIN', 'TOTAL_SNOW', 'DIRECTION_MAX_GUST',
       'SPEED_MAX_GUST', 'COOLING_DEGREE_DAYS', 'HEATING_DEGREE_DAYS',
       'nb_j_t_sup20_1w', 'nb_j_t_sup25_1w', 'nb_j_t_sup30_1w',
       'nb_j_t_inf_10_1w', 'nb_j_t_inf_0_1w', 'nb_jt__inf_m5_1w',
       'nb_j_t_inf_m10_1w', 'nb_j_precip_sup_5_1w', 'mean_t_4w', 'min_t_4w',
       'max_t_4w', 'tot_precip_4w', 'tot_rain_4w', 'tot_snow_4w',
       'nb_j_t_sup20_4w', 'nb_j_t_sup25_4w', 'nb_j_t_sup30_4w',
       'nb_j_t_inf_10_4w', 'nb_j_t_inf_0_4w', 'nb_jt__inf_m5_4w',
       'nb_j_t_inf_m10_4w', 'nb_j_precip_sup_5_4w', 'mean_t_3w', 'min_t_3w',
       'max_t_3w', 'tot_precip_3w', 'tot_rain_3w', 'tot_snow_3w',
       'nb_j_t_sup20_3w', 'nb_j_t_sup25_3w', 'nb_j_t_sup30_3w',
       'nb_j_t_inf_10_3w', 'nb_j_t_inf_0_3w', 'nb_jt__inf_m5_3w',
       'nb_j_t_inf_m10_3w', 'nb_j_precip_sup_5_3w', 'mean_t_2w', 'min_t_4w.1',
       'max_t_2w', 'tot_precip_2w', 'tot_rain_2w', 'tot_snow_2w',
       'nb_j_t_sup20_2w', 'nb_j_t_sup25_2w', 'nb_j_t_sup30_2w',
       'nb_j_t_inf_10_2w', 'nb_j_t_inf_0_2w', 'nb_jt__inf_m5_2w',
       'nb_j_t_inf_m10_2w', 'nb_j_precip_sup_5_2w']
X_FINAL = DF_FINAL[cols_X]

In [ ]:
X_FINAL.to_csv('X_final.csv', index=False)
Y_FINAL.to_csv('Y_final.csv', index=False)

In [ ]:
DF_final.columns